In [45]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import random
from transformers import BertTokenizer, BertModel
import json
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
torch.manual_seed(1)

In [46]:
device = "cpu"

In [39]:
#check if cuda is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

cuda


In [17]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

#### BiLSTM CRF model

In [43]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, batch_size=1, device='cpu'):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.target_size = target_size
        self.batch_size = batch_size
        self.device = device
        self.tag_to_ix = tag_to_ix
        self.start_tag = start_tag
        self.end_tag = end_tag

        self.embedding = nn.Embedding.from_pretrained(embedding_mat, freeze=False).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)

        self.transitions_to = nn.Parameter(torch.randn(target_size, target_size)).to(device)
        self.transitions_to.data[start_tag, :] = -10000
        self.transitions_to.data[:, end_tag] = -10000

        # self.transitions_from = nn.Parameter(torch.randn(target_size, target_size))
        # self.transitions_from.data[:, start_tag] = -10000
        # self.transitions_from.data[end_tag, :] = -10000

        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(self.device),
                torch.randn(2, 1, self.hidden_dim // 2).to(self.device))

    def get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.embedding(sentence).view(len(sentence), 1, -1)
        #convert embeds to torch float32
        embeds = embeds.float()
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats


    def _forward_algo(self, lstm_features):

        scores = torch.full((1, self.target_size), -10000.).to(self.device)
        scores[0][self.start_tag] = 0.

        forward_var = scores

        for feat in lstm_features:
            next_tag_var = self.transitions_to + feat.view(-1, 1).expand(-1, self.target_size) + forward_var.expand(self.target_size, -1)
            max_score = next_tag_var.max(dim=1).values.view(-1, 1)
            next_tag_var = next_tag_var - max_score
            forward_var = (max_score + torch.logsumexp(next_tag_var, dim=1).view(-1, 1)).view(1, -1)
            
        terminal_var = forward_var + (self.transitions_to[self.end_tag]).view(1, -1)
        alpha = terminal_var
        max_score = alpha.max()
        alpha = max_score + torch.logsumexp(alpha - max_score, dim=1)
        return alpha
    

    def _score_sentence(self, lstm_features, tags):
        score = torch.zeros(1).to(self.device)
        tags = torch.cat([torch.tensor([self.tag_to_ix['START_TAG']], dtype=torch.long).to(self.device), tags]).to(self.device)
        for i, feat in enumerate(lstm_features):
            score += self.transitions_to[tags[i + 1], tags[i]] + feat[tags[i + 1]]
                
        score += self.transitions_to[self.tag_to_ix['END_TAG'], tags[-1]]
        return score
    
    def neg_log_likelihood(self, sentence, tags):
        lstm_feats = self.get_lstm_features(sentence)
        forward_score = self._forward_algo(lstm_feats)
        gold_score = self._score_sentence(lstm_feats, tags)
        return forward_score - gold_score
    

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.target_size), -10000.).to(self.device)
        init_vvars[0][self.start_tag] = 0

        forward_var = init_vvars
        for feat in feats:
            bptrs_t = [] 
            viterbivars_t = [] 

            next_tag_var = self.transitions_to + forward_var.expand(self.target_size, -1)
            best_tag_id = torch.argmax(next_tag_var, dim=1)
            bptrs_t = best_tag_id
            viterbivars_t = next_tag_var[range(len(best_tag_id)), best_tag_id].view(1, -1)
            
            forward_var = (viterbivars_t + feat).view(1, -1)
            backpointers.append(bptrs_t)

        
        terminal_var = forward_var + self.transitions_to[self.end_tag]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

       
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id.item())
        
        start = best_path.pop()
        best_path.reverse()
        return path_score, best_path
        
    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self.get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

        

In [8]:
train_data = json.load(open('../Dataset/BIO_Tagged/ATE_train.json', 'r'))
test_data = json.load(open('../Dataset/BIO_Tagged/ATE_test.json', 'r'))
val_data = json.load(open('../Dataset/BIO_Tagged/ATE_val.json', 'r'))

In [10]:
word_to_idx = json.load(open('../Utils/word_to_idx.json', 'rb'))

In [11]:
tag_to_ix = json.load(open('../Utils/tag_to_ix.json', 'rb'))

#### Embeddings

In [13]:
bert_embedding_mat = pickle.load(open('../Extracted Word Embeddings/legal_bert_embedding_mat.pkl', 'rb'))
word2vec_embedding_mat = pickle.load(open('../Extracted Word Embeddings/word2vec_embedding_mat.pkl', 'rb'))
glove_embedding_mat = pickle.load(open('../Extracted Word Embeddings/glove_embedding_mat.pkl', 'rb'))

#### Glove

In [ ]:
model = BiLSTM_CRF(len(word_to_idx), 300, 256, len(tag_to_ix), torch.tensor(glove_embedding_mat).to(device), tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device=device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

EPOCHS = 10
train_loss = []
val_loss = []
train_macro_f1 = []
val_macro_f1 = []

for epoch in tqdm(range(EPOCHS), desc='Epoch'):
    loss = 0
    f1 = 0
    for i, case in tqdm(enumerate(train_data), desc=f'Epoch {epoch + 1}/{EPOCHS}'):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        model.train(True)
        model.zero_grad()
        loss = model.neg_log_likelihood(sentence, tags)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        with torch.no_grad():
            preds = model(sentence)[1]
            tags = tags.cpu().numpy()
            f1 += f1_score(tags, preds, average='macro')
    
    train_loss.append(loss/len(train_data))
    train_macro_f1.append(f1/len(train_data))
    
    model.eval()
    with torch.no_grad():
        loss = 0
        f1 = 0
        for case in val_data:
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            tags = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            setence = sentence.to(device)
            tags = tags.to(device)
            loss = model.neg_log_likelihood(sentence, tags)
            loss += loss.item()
            preds = model(sentence)[1]
            f1 += f1_score(tags.cpu().numpy(), preds, average='macro')
        val_macro_f1.append(f1/len(val_data))
        val_loss.append(loss/len(val_data))

    print()
    print(f'Train loss: {train_loss[-1]}, Val loss: {val_loss[-1]}')
    print(f'Train macro f1: {train_macro_f1[-1]}, Val macro f1: {val_macro_f1[-1]}')
    print(f'Epoch {epoch + 1}/{EPOCHS} done')

In [ ]:
torch.save(model, 'Non Trainable Embeddings/Glove/bilstm_crf_glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove/val_loss.pkl', 'wb'))
pickle.dump(train_macro_f1, open('Non Trainable Embeddings/Glove/train_f1.pkl', 'wb'))
pickle.dump(val_macro_f1, open('Non Trainable Embeddings/Glove/val_f1.pkl', 'wb'))

In [ ]:
model.eval()
with torch.no_grad():
    f1 = 0
    for case in test_data:
        sentence = prepare_sequence(test_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(test_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        preds = model(sentence)[1]
        tags = tags.cpu().numpy()
        f1 += f1_score(tags, preds, average='macro')
    print(f'Glove + BiLSTEM-CRF Test macro f1: {f1/len(test_data)}')

#### Word2vec

In [ ]:
model = BiLSTM_CRF(len(word_to_idx), 300, 256, len(tag_to_ix), torch.tensor(word2vec_embedding_mat).to(device), tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device=device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

EPOCHS = 10
train_loss = []
val_loss = []
train_macro_f1 = []
val_macro_f1 = []

for epoch in tqdm(range(EPOCHS), desc='Epoch'):
    loss = 0
    f1 = 0
    for i, case in tqdm(enumerate(train_data), desc=f'Epoch {epoch + 1}/{EPOCHS}'):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        model.train(True)
        model.zero_grad()
        loss = model.neg_log_likelihood(sentence, tags)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        with torch.no_grad():
            preds = model(sentence)[1]
            tags = tags.cpu().numpy()
            f1 += f1_score(tags, preds, average='macro')
    
    train_loss.append(loss/len(train_data))
    train_macro_f1.append(f1/len(train_data))
    
    model.eval()
    with torch.no_grad():
        loss = 0
        f1 = 0
        for case in val_data:
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            tags = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            setence = sentence.to(device)
            tags = tags.to(device)
            loss = model.neg_log_likelihood(sentence, tags)
            loss += loss.item()
            preds = model(sentence)[1]
            f1 += f1_score(tags.cpu().numpy(), preds, average='macro')
        val_macro_f1.append(f1/len(val_data))
        val_loss.append(loss/len(val_data))

    print()
    print(f'Train loss: {train_loss[-1]}, Val loss: {val_loss[-1]}')
    print(f'Train macro f1: {train_macro_f1[-1]}, Val macro f1: {val_macro_f1[-1]}')
    print(f'Epoch {epoch + 1}/{EPOCHS} done')

In [ ]:
torch.save(model, 'Non Trainable Embeddings/Word2vec/bilstm_crf_word2vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec/val_loss.pkl', 'wb'))
pickle.dump(train_macro_f1, open('Non Trainable Embeddings/Word2vec/train_f1.pkl', 'wb'))
pickle.dump(val_macro_f1, open('Non Trainable Embeddings/Word2vec/val_f1.pkl', 'wb'))

In [ ]:
model.eval()
with torch.no_grad():
    f1 = 0
    for case in test_data:
        sentence = prepare_sequence(test_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(test_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        preds = model(sentence)[1]
        tags = tags.cpu().numpy()
        f1 += f1_score(tags, preds, average='macro')
    print(f'Word2Vec + BiLSTEM-CRF Test macro f1: {f1/len(test_data)}')

#### Bert

In [ ]:
model = BiLSTM_CRF(len(word_to_idx), 768, 512, len(tag_to_ix), torch.tensor(bert_embedding_mat).to(device), tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device=device).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

EPOCHS = 10
train_loss = []
val_loss = []
train_macro_f1 = []
val_macro_f1 = []

for epoch in tqdm(range(EPOCHS), desc='Epoch'):
    loss = 0
    f1 = 0
    for i, case in tqdm(enumerate(train_data), desc=f'Epoch {epoch + 1}/{EPOCHS}'):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        model.train(True)
        model.zero_grad()
        loss = model.neg_log_likelihood(sentence, tags)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        with torch.no_grad():
            preds = model(sentence)[1]
            tags = tags.cpu().numpy()
            f1 += f1_score(tags, preds, average='macro')
    
    train_loss.append(loss/len(train_data))
    train_macro_f1.append(f1/len(train_data))
    
    model.eval()
    with torch.no_grad():
        loss = 0
        f1 = 0
        for case in val_data:
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            tags = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            setence = sentence.to(device)
            tags = tags.to(device)
            loss = model.neg_log_likelihood(sentence, tags)
            loss += loss.item()
            preds = model(sentence)[1]
            f1 += f1_score(tags.cpu().numpy(), preds, average='macro')
        val_macro_f1.append(f1/len(val_data))
        val_loss.append(loss/len(val_data))

    print()
    print(f'Train loss: {train_loss[-1]}, Val loss: {val_loss[-1]}')
    print(f'Train macro f1: {train_macro_f1[-1]}, Val macro f1: {val_macro_f1[-1]}')
    print(f'Epoch {epoch + 1}/{EPOCHS} done')

In [ ]:
torch.save(model, 'Non Trainable Embeddings/Bert/bilstm_crf_bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert/val_loss.pkl', 'wb'))
pickle.dump(train_macro_f1, open('Non Trainable Embeddings/Bert/train_f1.pkl', 'wb'))
pickle.dump(val_macro_f1, open('Non Trainable Embeddings/Bert/val_f1.pkl', 'wb'))

In [54]:
model.eval()
with torch.no_grad():
    f1 = 0
    for case in test_data:
        sentence = prepare_sequence(test_data[case]['text'].split(' '), word_to_idx)
        tags = prepare_sequence(test_data[case]['labels'], tag_to_ix)
        sentence = sentence.to(device)
        tags = tags.to(device)
        preds = model(sentence)[1]
        tags = tags.cpu().numpy()
        f1 += f1_score(tags, preds, average='macro')
    print(f'Bert + BiLSTEM-CRF Test macro f1: {f1/len(test_data)}')

Test macro f1: 0.771086573696499
